original code
https://www.kaggle.com/code/emiz6413/training-gemma-2-9b-4-bit-qlora-fine-tuning

Suggestions https://www.kaggle.com/competitions/lmsys-chatbot-arena/discussion/527596

Use TTA! Change tta = False to tta = True boosts LB 0.941 => 0.926<br>
Add modules: ["down_proj","up_proj","o_proj","gate_proj"] and use r=64, a=4 boost LB 0.913 => 0.903<br>
Train with max=2048 vs max=1024 boost LB 0.899 => 0.895<br>
Change LoRA r=64, a=4 to r=1024, a=4 boost LB 0.895 => 0.894<br>
Truncate from left vs right boost LB 0.890 => 0.885<br>

# Decrease validation size

In [ ]:
# gemma-2 is available from transformers>=4.42.3
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

In [ ]:
# import zipfile
# import os

# # Path to the zip file and destination folder
# zip_file_path = '/content/checkpoint-5748.zip'
# destination_folder = '/content/checkpoint-5748/'

# # Create the destination folder if it doesn't exist
# os.makedirs(destination_folder, exist_ok=True)

# # Unzip the file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(destination_folder)

# print(f"Unzipped files to {destination_folder}")


In [ ]:
# THIS CELL IS VERY IMPORTANT VERIFY IT THROUGHLY
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 2048
    n_splits: int = 10  # Experimented on kaggle increasing this will increase the traindata size and reduce eval
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2   ### INCREASE THIS
    gradient_accumulation_steps: int = 1  # global batch size is 8
    per_device_eval_batch_size: int = 2
    n_epochs: int = 1             ## EPOCHS HERE
    freeze_layers: int = 12  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 1024  #CHANGED FROM 16 TO 1024
    lora_alpha: float = 4
    lora_dropout: float = 0.05
    lora_bias: str = "none"

config = Config()

In [ ]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=500,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules = ["q_proj", "k_proj", "v_proj", "down_proj","up_proj","o_proj","gate_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

In [ ]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# # LEARNED HOW TO START TRAINING FROM THE PREVIOUS CHECKPOINT
# from peft import PeftModel
# model = PeftModel.from_pretrained(model, Config.lora_dir)

model

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-11): 12 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
            )
            (mlp): Gemma2MLP(
              (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
              (act_fn): PytorchGELUTanh()
 

In [ ]:
model.print_trainable_parameters()

trainable params: 2,469,407,232 || all params: 11,711,123,968 || trainable%: 21.0860


In [ ]:
import pandas as pd
extra_data1 = pd.read_csv('/content/lmsys-33k-deduplicated.csv')
extra_data1.head(5)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,"[""Why did my parent not invite me to their wed...","[""It is possible that your parent did not invi...","[""It is likely that they wanted to keep the gu...",0,0,1
1,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,"[""How to build an arena for chatbots?""]","[""Building an arena for chatbots can be done b...","[""Building an arena for chatbots is a great wa...",0,1,0
2,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,"[""When is it today?""]","[""I'm sorry, I cannot determine the current da...","[""Today is February 23, 2023.""]",1,0,0
3,c0fc42c6f5f14f2aa5a89f71f8553730,vicuna-13b,koala-13b,"[""Count from 1 to 10 with step = 3""]","[""1, 4, 7, 10\n\nCounting with a step of 3 mea...","[""1, 4, 7, 10""]",1,0,0
4,c4938f25c1d94fc1b110ace95a2243d0,vicuna-13b,koala-13b,"[""Emoji for \""sharing\"". List 10""]","[""1. \ud83d\udcaa Sharing progress or achievem...","[""1. Sharing hands\n* Share icon\n* Grou...",1,0,0


In [ ]:
extra_dataset2 = pd.read_csv('/content/lmsys-33k.csv')
extra_dataset2.head(5)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,58210e39b3fd4441a2bd4a518bb44c2d,chatglm-6b,koala-13b,"[""What is the difference between OpenCL and CU...","[""OpenCL and CUDA are two different programmin...","[""OpenCL and CUDA are both programming languag...",0,1,0
1,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,"[""Why did my parent not invite me to their wed...","[""It is possible that your parent did not invi...","[""It is likely that they wanted to keep the gu...",0,0,1
2,90bfd142157948aba01931726c888e7f,koala-13b,oasst-pythia-12b,"[""Fuji vs. Nikon, which is better?""]","[""Both Fuji and Nikon are popular camera brand...","[""This is a subjective question and the answer...",0,1,0
3,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,"[""How to build an arena for chatbots?""]","[""Building an arena for chatbots can be done b...","[""Building an arena for chatbots is a great wa...",0,1,0
4,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,"[""When is it today?""]","[""I'm sorry, I cannot determine the current da...","[""Today is February 23, 2023.""]",1,0,0


In [ ]:
import pandas as pd
input_columns = ['prompt' , 'response_a','response_b']
label_columns = ['winner_model_a','winner_model_b']

raw_train_dataset = pd.read_parquet('/content/train.parquet')
def to_onehot(row):
    # Return the row with new columns added
    return pd.Series({
        "winner_model_a": 1 if row['winner'] == "model_a" else 0,
        "winner_model_b": 1 if row['winner'] == "model_b" else 0
    })

# Apply and add new columns
onehot_encoded = raw_train_dataset.apply(to_onehot, axis=1)

# Add the one-hot encoded columns to the original DataFrame
raw_train_dataset = pd.concat([raw_train_dataset, onehot_encoded], axis=1)
raw_train_dataset = raw_train_dataset.dropna().drop(['language'],axis=1).reset_index(drop=True)
raw_train_dataset['winner_tie'] = pd.Series([0] * raw_train_dataset.shape[0])
raw_train_dataset = raw_train_dataset[['id','model_a','model_b','prompt','response_a','response_b','winner_model_a','winner_model_b','winner_tie']]

In [ ]:
raw_train_dataset.head(5)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,o1-preview,reka-core-20240904,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",1,0,0
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,gemma-2-27b-it,gemini-1.5-flash-002,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",1,0,0
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,gpt-4-0125-preview,claude-3-5-sonnet-20240620,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,1,0,0
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,0,1,0
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,reka-flash-20240722,grok-2-2024-08-13,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",1,0,0


In [ ]:
# Optionally inspect the structure to ensure compatibility
print("Dataset 1:", raw_train_dataset.columns)
print("Dataset 2:", extra_data1.columns)
print("Dataset 3:", extra_dataset2.columns)

# Align schemas if necessary (e.g., rename columns or add missing columns)
# Ensure all datasets have the same column names for concatenation

# Concatenate the datasets
combined_dataset = pd.concat([raw_train_dataset, extra_data1, extra_dataset2])
combined_dataset = combined_dataset.drop(['id','model_a','model_b'],axis=1)

Dataset 1: Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')
Dataset 2: Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')
Dataset 3: Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')


In [ ]:
print(combined_dataset.shape)
combined_dataset.head(5)

(102626, 6)


,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",1,0,0
1,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",1,0,0
2,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,1,0,0
3,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,0,1,0
4,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",1,0,0


In [ ]:
from datasets import Dataset
ds = Dataset.from_pandas(combined_dataset)
ds

Dataset({
    features: ['prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie', '__index_level_0__'],
    num_rows: 102626
})

In [ ]:
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Set tokenizer to truncate from the left
        self.tokenizer.truncation_side = "left"

    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]

        # Tokenize with truncation
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)

        # Create labels
        labels = []
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)

        return {**tokenized, "labels": labels}

    @staticmethod
    def process_text(text: str) -> str:
        """
        Safely process text to handle multilingual and arbitrary input.
        """
        if text is None:
            return ""  # Handle null inputs
        try:
            return " ".join(str(text).split())  # Remove excess whitespace
        except Exception as e:
            print(f"Error processing text: {text}. Error: {e}")
            return ""  # Return empty string for invalid inputs


In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

Map:   0%|          | 0/102626 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

In [ ]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    )
    for fold_idx in range(config.n_splits)
]

In [ ]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

<ipython-input-22-ac08366e6a3d>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
# Define directories
base_model_dir = "saved_base_model"
lora_adapter_dir = "saved_lora_adapter"
tokenizer_dir = "saved_tokenizer"

# Save the base model
model.save_pretrained(base_model_dir)

# Save the LoRA adapter
# model.save_adapter(lora_adapter_dir)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_dir)

print("Base model, LoRA adapter, and tokenizer saved successfully!")


Base model, LoRA adapter, and tokenizer saved successfully!


In [ ]:
# import shutil

# # Path to the folder to be zipped
# folder_path = '/content/output/checkpoint-11500'
# zip_file_path = 'checkpoint-750'

# # Create a zip file of the folder
# shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)